In [77]:
from time import sleep
from os import path
import googleapiclient.discovery
import json
import os

In [4]:
DEVELOPER_KEY = "KEY"

In [5]:
def yt_videos_list(ids):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.videos().list(
        part="snippet,contentDetails,topicDetails,statistics,liveStreamingDetails",
        maxResults=50,
        id=ids
    )
    response = request.execute()
    
    return response

In [78]:
# load list of ids
with open('../../download/ids.txt', 'rb') as f:
    video_ids = f.read()
    
video_ids = video_ids.decode("utf-16").rstrip("\r\n").split("\r\n") # ref: https://stackoverflow.com/a/45717703

In [52]:
chunks = []

for i in range(1 + (len(video_ids) // 50)):
    chunk = video_ids[i*50:(i+1)*50]
    
    if len(chunk) > 0:
        chunks.append(chunk)

In [54]:
res = []

for i, chunk in enumerate(chunks):
    print('Getting chunk %i' % (i))
    res.append(yt_videos_list(','.join(chunk)))
    sleep(3)

Getting chunk 0
Getting chunk 1
Getting chunk 2
Getting chunk 3
Getting chunk 4
Getting chunk 5
Getting chunk 6
Getting chunk 7
Getting chunk 8
Getting chunk 9
Getting chunk 10
Getting chunk 11
Getting chunk 12
Getting chunk 13
Getting chunk 14
Getting chunk 15
Getting chunk 16
Getting chunk 17
Getting chunk 18
Getting chunk 19
Getting chunk 20
Getting chunk 21
Getting chunk 22
Getting chunk 23
Getting chunk 24
Getting chunk 25
Getting chunk 26
Getting chunk 27
Getting chunk 28
Getting chunk 29
Getting chunk 30
Getting chunk 31
Getting chunk 32
Getting chunk 33
Getting chunk 34


In [55]:
def filter_items(data):
    """ Extract items and remove duplicates """
    seen = set()
    items = []

    for page in data:
        for item in page['items']:
            item_id = str(item['id'])

            if not item_id in seen:
                items.append(item)
                seen.add(item_id)
    
    return items

In [82]:
items = filter_items(res)

In [83]:
assert set(video_ids) - set([x['id'] for x in items]) == set()

In [75]:
with open('videos_list.json', 'w') as f:
    f.write(json.dumps(items))